In [130]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.autograd import Variable
from sklearn.datasets import load_iris
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import random

In [131]:

## making custom dataset class for iris

class IRISDataSet(Dataset):

    def __init__(self, data, labels, transform=None) -> None:
        super(Dataset).__init__()
        self.data = data
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, index):
        if self.transform:
            sample = self.transform(self.data[index])
        else:
            sample = self.data[index]
        label = self.labels[index]
        return sample, label
    
    def train_test_split(self, split_ratio=0.7):
        
        shuffl_idx = list(range(len(self)))
        random.shuffle(shuffl_idx)
        split_idx = int(split_ratio*len(shuffl_idx))
        train_dataset = IRISDataSet(self.data[:split_idx], 
                                    self.labels[:split_idx], self.transform)
        test_dataset = IRISDataSet(self.data[split_idx:],
                                   self.labels[split_idx:], self.transform)
        
        return train_dataset, test_dataset

# transform = transforms.Compose([transforms.ToTensor()])


In [132]:
## creating a Neural Network Class

class NNet(nn.Module):

    def __init__(self, input_size:int, hidden_layers:list, num_classes) -> None:
        super(NNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_layers[0])
        self.relu1 = nn.ReLU()
        self.hidden_layers = nn.ModuleList()
        self.hidden_relu = nn.ModuleList()
        for i in range(len(hidden_layers)-1):
            self.hidden_layers.append(nn.Linear(hidden_layers[i], hidden_layers[i+1]))
            self.hidden_relu.append(nn.ReLU())

        self.l_out = nn.Linear(hidden_layers[-1], num_classes)

    def forward(self, x):
        x = self.l1(x)
        x = self.relu1(x)
        for hidden, relu in zip(self.hidden_layers, self.hidden_relu):
            x = hidden(x)
            x = relu(x)

        out = self.l_out(x)
        return out



In [133]:
##data preperation

iris = load_iris(as_frame=True)
data = iris.data
X = torch.from_numpy(data[:].values) 
y = torch.from_numpy(iris.target[:].values)
# print(y, y.shape)
# y = y.reshape((-1,1))

dataset = IRISDataSet(X, y)

In [134]:
## splitting into train and test
train_dataset, test_dataset = dataset.train_test_split()

In [135]:
## dataloaders
batch_size = 8
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [136]:
## device to use
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [137]:
## instatiating an object of NNet

model = NNet(4, [10, 10], 3).to(device)
print(model)
# x = torch.randn(8,4)
# print(model(x).shape)

NNet(
  (l1): Linear(in_features=4, out_features=10, bias=True)
  (relu1): ReLU()
  (hidden_layers): ModuleList(
    (0): Linear(in_features=10, out_features=10, bias=True)
  )
  (hidden_relu): ModuleList(
    (0): ReLU()
  )
  (l_out): Linear(in_features=10, out_features=3, bias=True)
)


In [138]:
creterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [143]:
n_epochs = 5
for epoch in range(n_epochs):
    num_correct = 0
    num_samples = 0
    for batch_idx, (data, targets) in enumerate(train_dataloader):
        
        
        # Get data and targets to cuda if available
        data = data.to(device, dtype=torch.float32)
        targets = targets.to(device)

        #forward
        scores = model(data)
        # print(scores.shape)
        # print(targets.shape)
        loss = creterion(scores, targets)

        # check accuracy
        _,preds = scores.max(1)
        # print(preds.shape)
        # print(y.shape)
        num_correct += (preds == targets).sum()
        num_samples += preds.size(0)


        # backward
        optimizer.zero_grad()
        loss.backward()

        # update
        optimizer.step()
    acc_train = num_correct/num_samples

    # calculate accuracy on validation data
    model.eval()
    with torch.no_grad():
        
        for data, targets in val_dataloader:

            data = data.to(device, dtype=torch.float32)
            targets = targets.to(device)
            scores = model(data)
            _,preds = scores.max(1)
            num_correct += (preds == targets).sum()
            num_samples += preds.size(0)
    model.train()
    acc_val = num_correct/ num_samples

    print(f'Epoch: {epoch + 1}\t Training Accuracy: {acc_train:0.2f}\t Val. Accuracy: {acc_val:0.2f}.')


Epoch: 1	 Training Accuracy: 0.95	 Val. Accuracy: 0.00.
Epoch: 2	 Training Accuracy: 0.95	 Val. Accuracy: 0.00.
Epoch: 3	 Training Accuracy: 0.95	 Val. Accuracy: 0.00.
Epoch: 4	 Training Accuracy: 0.95	 Val. Accuracy: 0.00.
Epoch: 5	 Training Accuracy: 0.95	 Val. Accuracy: 0.00.
